<a href="https://colab.research.google.com/github/3bbinesh/ML_NLP/blob/main/NLPWithBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import AutoTokenizer, BertModel
from torch.utils.data import TensorDataset, DataLoader
import sys
import torch
import numpy as np

In [23]:
model = BertModel.from_pretrained("bert-base-uncased")
#sentence1 = sys.argv[1]  # First sentence
#sentence2 = sys.argv[2]  # Second sentence
sentence1 = "The city of joy"
sentence2 = "better known as Kolkata"
sentences = [sentence1, sentence2]

In [25]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
sentences_tokenized = [tokenizer.tokenize(s) for s in sentences]
MAX_LENGTH = max([len(sentences_tokenized[0]), len(sentences_tokenized[1])])

In [26]:
# Generate IDs of each token and add padding to sentences smaller than given threshold
ids = [tokenizer.convert_tokens_to_ids(t) for t in sentences_tokenized]
# Pad the tokens
ids = np.asarray([np.pad(i, (0, MAX_LENGTH-len(i)), mode='constant') for i in ids])

In [27]:
# Generate the masks
amasks = np.asarray([[float(i>0) for i in seq] for seq in ids])

# Get the output from the model
output = model.forward(torch.tensor(ids), torch.tensor(amasks))

# Now create two vector representations for each sentence
# pool_vectors (2x768) is the vector generated by perofrming max-pooling over the hidden states
pool_vectors = torch.max(output.last_hidden_state, dim=1)[0]
# cls_vectors (2x768) is the vector generated by taking the CLS token
cls_vectors = output.last_hidden_state[:, 0, :]

In [14]:
def cosine_similarity(vec1, vec2):
    dot_product = torch.dot(vec1, vec2)
    norm1 = torch.norm(vec1)
    norm2 = torch.norm(vec2)
    cosine_sim = dot_product / (norm1 * norm2)
    return cosine_sim

In [28]:
#cosine_pooling = --------------       # Fill up this '----------' section
cosine_pooling = cosine_similarity(pool_vectors[0], pool_vectors[1])
#cosine_cls = ------------------       # Fill up this '----------' section
cosine_cls = cosine_similarity(cls_vectors[0], cls_vectors[1])

# Finally print out the values
print(np.round(cosine_pooling.item(), 2), np.round(cosine_cls.item(), 2))

0.51 0.3


In [16]:

    # Tokenize each sentence
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
    sentences_tokenized = [tokenizer.tokenize(s) for s in sentences]
    MAX_LENGTH = max([len(sentences_tokenized[0]), len(sentences_tokenized[1])])

    # Generate IDs of each token and add padding to sentences smaller than given threshold
    ids = [tokenizer.convert_tokens_to_ids(t) for t in sentences_tokenized]
    # Pad the tokens
    ids = np.asarray([np.pad(i, (0, MAX_LENGTH-len(i)), mode='constant') for i in ids])


    # Generate the masks
    amasks = np.asarray([[float(i>0) for i in seq] for seq in ids])

    # Get the output from the model
    output = model.forward(torch.tensor(ids), torch.tensor(amasks))

    # Now create two vector representations for each sentence
    # pool_vectors (2x768) is the vector generated by perofrming max-pooling over the hidden states
    pool_vectors = torch.max(output.last_hidden_state, dim=1)[0]
    # cls_vectors (2x768) is the vector generated by taking the CLS token
    cls_vectors = output.last_hidden_state[:, 0, :]

    # Compute the cosine similarity
    def cosine_similarity(vec1, vec2):
        dot_product = torch.dot(vec1, vec2)
        norm1 = torch.norm(vec1)
        norm2 = torch.norm(vec2)
        cosine_sim = dot_product / (norm1 * norm2)
        return cosine_sim
    #cosine_pooling = --------------       # Fill up this '----------' section
    cosine_pooling = cosine_similarity(pool_vectors[0], pool_vectors[1])
    #cosine_cls = ------------------       # Fill up this '----------' section
    cosine_cls = cosine_similarity(cls_vectors[0], cls_vectors[1])

    # Finally print out the values
    print(np.round(cosine_pooling.item(), 2), np.round(cosine_cls.item(), 2))

0.89 0.85
